<a href="https://colab.research.google.com/github/CristianMorasso/statistical_project/blob/master/Copia_di_statiscal_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Learning Project: ALFA Dataset Anaysis


In [35]:
!pip install icecream sktime scikit-learn polars numpy
!rm -rf statiscal_project
!git clone https://github.com/CristianMorasso/statistical_project.git
!pwd
# # Connect Drive to get the Dataset
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)
# dataset_dir= '/content/drive/MyDrive/alfa_dataset'


fatal: destination path 'statistical_project' already exists and is not an empty directory.
/content


In [36]:
dataset_dir = "statistical_project/data"

In [108]:
# Read the data from the csv files
import numpy as np
import pandas  as pd
import os

dfs = [] # dataframe list
class_dict = {}

class_id = 0
y_time_series = []

for idx,cl  in enumerate(os.listdir(dataset_dir)):
  if idx == 6: break
  path1 = os.path.join(dataset_dir, cl)
  if not os.path.isdir(path1) or cl == "features":
    print(f"{cl} is not a dir or not useful!")
    continue
  class_dict[class_id] = cl
  print(f"Class: {cl}")
  for f in  os.listdir(path1):
    print(f"\t File:{f}")
    pathf = os.path.join(path1, f)
    df = pd.read_csv(pathf)

    # Remove Label field
    label_field = "field.data"
    try:
      df = df.drop(label_field, axis=1)
    except:
      print(f"\t- The {label_field}~ is not in the {f} file")

    # Save the id for the serie
    y_time_series.append(class_id)

    # This would be useless but the fit algorithm needs this
    df.insert(0, "Y", [ class_id for _ in range(len(df))], True)
    dfs.append(df)

  class_id+=1
  print() # Skip line

y_time_series = np.array(y_time_series)

# Checks
from collections import Counter
print(f"Y: {Counter(list(map(lambda x: class_dict[x], y_time_series)))}")

print(f"\nNumber of Time Series: {len(dfs)}")
#assert len(dfs) == 46  # We know this from the repo

In [109]:
class_dict


{0: 'no_failure', 1: 'elevator', 2: 'engine'}

## Feature selection

In [110]:
print("Before: ")
for df in dfs:
  print(df.shape)

# Only use the variables that can be found in all the dataframes
features = []
# Get the biggest set:
# (We are considering that that the bigger set is just a superset and has all the other base features)
for df in dfs:
   features = max(features, df.columns.values, key=lambda x: len(x))
# Then get the set intersection of all of the features sets
features = set(features)
for df in dfs:
  features = features.intersection(set(df.columns.values))

# Apply the feature mask to all the datframe in our dataset
features = list(features)
dfs = list(map(lambda x: x[features],dfs))

print("\nAfter: ")
for df in dfs:
  print(df.shape)

In [111]:
from sklearn.feature_selection import SelectKBest

# Concat all the time series in as single dataframe
concatted_dfs = pd.concat(dfs)
y = concatted_dfs["Y"]

# Now remove it again from the concatted dataframe
concatted_dfs = concatted_dfs.drop("Y", axis=1)
# Also remove the time column!
concatted_dfs = concatted_dfs.drop("%time", axis=1)

# Print the number of nans per per each columns
print(concatted_dfs.isnull().sum())
print(f"Length of Concatted Dataframes: {len(concatted_dfs)}")

In [112]:
# Inpute the nan values to the mean of the other entries value of the same column
from sklearn.impute import SimpleImputer
import numpy as np

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(concatted_dfs)
inp_out = imp.transform(concatted_dfs)

In [113]:
# Get the best 20 features of the dataset
columns_ = concatted_dfs.columns.values
feature_mask= SelectKBest(k=20).fit(inp_out, y).get_feature_names_out(columns_)
# Uses: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif

print("Feature Mask:")
print(feature_mask)
print(f"Length of Feature Mask: {len(feature_mask)}")

In [114]:
# Now finally use the mask on every dataframe in the original dataframeList
# and also inpute but using time-series specific data:
new_dfs = list(map(lambda x: SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x[feature_mask]) , dfs))

# Now we have the 20 best features without any nan value:
type(new_dfs[0]) # is a numpy.ndarray! wow!
print(new_dfs[0][:2,:]) # First 2 lines
print(new_dfs[0].shape) # shape of the array

In [115]:
# Now finally use the mask on every dataframe in the original
# dataframeList and also inpute but using time-series specific data:
new_dfs = list(map(lambda x: SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x[feature_mask]) , dfs))

# Now we have the 20 best features without any nan value:
type(new_dfs[0]) # is a numpy.ndarray! wow!
print(new_dfs[0][:2,:]) # First 2 lines
print(new_dfs[0].shape) # shape of the array

In [116]:
# Finally let's create a multiIndexed pandas.Dataframe so that we can use our data with the sktime library
_dfs = []
for i, npVals in enumerate(new_dfs):
  df = pd.DataFrame(npVals,columns=feature_mask)
  df.insert(0, "flight", [ i for _ in range(len(df))], True)
  df.insert(1, "step", [ a for a in range(len(df))], True)
  _dfs.append(df)
dfs = pd.concat(_dfs)
print(dfs.shape)
dfs = dfs.drop_duplicates(subset=['flight', 'step'], keep='last')
dfs = dfs.set_index(["flight", "step"])
print(dfs.shape)



# Testing Models:

In [117]:
# list all classifiers in sktime
from sktime.registry import all_estimators


all_estimators("classifier", as_dataframe=True) # , filter_tags={"capability:unequal_length":TrueNN

,name,object
0,Arsenal,<class 'sktime.classification.kernel_based._ar...
1,BOSSEnsemble,<class 'sktime.classification.dictionary_based...
2,BOSSVSClassifierPyts,<class 'sktime.classification.dictionary_based...
3,BaggingClassifier,<class 'sktime.classification.ensemble._baggin...
4,CNNClassifier,<class 'sktime.classification.deep_learning.cn...
5,CNTCClassifier,<class 'sktime.classification.deep_learning.cn...
6,CanonicalIntervalForest,<class 'sktime.classification.interval_based._...
7,Catch22Classifier,<class 'sktime.classification.feature_based._c...
8,ClassifierPipeline,<class 'sktime.classification.compose._pipelin...
9,ColumnEnsembleClassifier,<class 'sktime.classification.compose._column_...


In [118]:
from sktime.datatypes import MTYPE_REGISTER
pd.DataFrame(MTYPE_REGISTER)

,0,1,2
0,pd.Series,Series,pd.Series representation of a univariate series
1,pd.DataFrame,Series,pd.DataFrame representation of a uni- or multi...
2,np.ndarray,Series,"2D numpy.ndarray with rows=samples, cols=varia..."
3,xr.DataArray,Series,xr.DataArray representation of a uni- or multi...
4,dask_series,Series,xdas representation of a uni- or multivariate ...
5,nested_univ,Panel,"pd.DataFrame with one column per variable, pd...."
6,numpy3D,Panel,"3D np.array of format (n_instances, n_columns,..."
7,numpyflat,Panel,"WARNING: only for internal use, not a fully su..."
8,pd-multiindex,Panel,"pd.DataFrame with multi-index (instances, time..."
9,pd-wide,Panel,"pd.DataFrame in wide format, cols = (instance*..."


In [119]:
from sktime.split import temporal_train_test_split
from sktime.datatypes import check_raise
from icecream import ic

check_raise(dfs, mtype="pd-multiindex")
check_raise(y_time_series, mtype='np.ndarray')

dfs =  PaddingTransformer().fit_transform(dfs)

y_train, y_test, X_train, X_test  = temporal_train_test_split(X=dfs,y=y_time_series, test_size=10)
ic(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_test.head()

#X_train.head()

ic| X_train.shape: (78425, 20)
    y_train.shape: (25, 1)
    X_test.shape: (31370, 20)
    y_test.shape: (10, 1)


0     1         2          3     4         5         6   \
flight step                                                                   
25     0     10.464746  15.0 -0.049067 -40.663937  0.03  0.015873 -0.063166   
       1     12.464252  15.0 -0.001174 -39.386497  0.00  0.015873 -0.027216   
       2     12.464252  15.0 -0.001174 -39.386497  0.00  0.015873 -0.027216   
       3     12.464252  15.0 -0.001174 -39.386497  0.00  0.015873 -0.027216   
       4     12.464252  15.0 -0.001174 -38.050343 -0.07  0.015873 -0.027216   

                   7           8        9         10        11        12  \
flight step                                                                
25     0    -1.571506 -272.974579  1.62692 -0.081678 -0.044222 -1.877345   
       1    -1.571506 -272.974579  1.62692  0.032931 -0.044222 -2.067345   
       2    -1.486565 -272.974579  1.62692  0.032931 -0.044222 -2.067345   
       3    -1.460196 -272.974579  1.62692  0.032931 -0.044222 -2.067345   
       4    -1.354021 -267.455688  1.62692  0.032931 -0.044222 -2.287345   

                   13        14        15        16         17        18  \
flight step                                                                
25     0     0.454218  0.370956  0.654647  0.572008  12.967053 -0.063446   
       1     0.454218 -0.039227  0.892405  0.572008  12.967053 -0.003525   
       2     0.454218 -0.039227  0.892405  0.710144  12.976702 -0.003525   
       3     0.454218 -0.039227  0.892405  0.741011  12.964842 -0.003525   
       4     0.454218 -0.039227  0.892405  0.920593  12.964248 -0.003525   

                     19  
flight step              
25     0     190.658913  
       1     190.658913  
       2     190.658913  
       3     190.658913  
       4     190.658913

In [ ]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.feature_based import RandomIntervalClassifier
from sktime.classification.deep_learning.cnn import CNNClassifier

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.padder import PaddingTransformer
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.transformations.panel.interpolate import TSInterpolator

cnn = CNNClassifier(n_epochs=2000, batch_size=4, kernel_size=3, avg_pool_size=3, n_conv_layers=3, verbose=True, loss='binary_crossentropy')
cnn.fit(X_train, y_train)
cnn.score(X_test,y_test)

#clf = Pipeline([
#    ("transform", TSInterpolator(50)),
#     ("concatenate", ColumnConcatenator()),
#     ("classify", TimeSeriesForestClassifier(n_estimators=100)),
#    ("transform", PaddingTransformer()),
#    ("classify", RandomIntervalClassifier(n_intervals=10)),
#])
#clf.fit(X_train, y_train)
#clf.score(X_test,y_test)
#

In [ ]:
from sktime.classification.deep_learning.inceptiontime import InceptionTimeClassifier
network = InceptionTimeClassifier(n_epochs=75, verbose=False)
network.fit(X_train, y_train)
network.score(X_test, y_test)